# Predicting Customer Churn With Logistic Regression

* Analysis performed using purely PySpark;
* Churn dataset with 10,000 instances. Instances are randomly split, 70% for training and 30% for testing;
* Two scenarios evaluated: one without compensating for class imbalance and one compensating for it;
* Class imbalance dealt with using class weights in logistic regression;
* Two models created, one for each scenario. Models evaluated using accuracy, weighted precision, weighted recall, AUCROC, AUCPR and F1-score;
* Dataset available online at: https://www.kaggle.com/code/mathchi/churn-problem-for-bank-customer

## Dataset Summary

The dataset is composed of information from bank customers with the response variable "Exited" indicating whether the customer left the bank or not. The table below provides some details on each one of the attributes.

| Attribute | Summary |
|:---------:|:-------:|
| CreditScore | Credit score calculated by the bank. |
| Geography | Country where the customer resides: Germany, France or Spain. |
| Gender | Gender between Female and Male. |
| Age | Customer's age. |
| Tenure | Amount of years the customer has been with the bank. |
| Balance | Current balance. |
| NumOfProducts | Number of bank products used. |
| HasCrCard | Credit card status (0: does not have; 1: has credit card). |
| IsActiveMember | Active membership status (0: not active; 1: active). |
| EstimatedSalary | Customer's estimated salary. |
| Exited | Customer status (0: has not left the bank; 1: has left the bank). |


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import round, udf
from pyspark.sql.types import DoubleType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import RFormula, VectorAssembler, StringIndexer
from pyspark.ml.stat import Correlation
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

## Data Exploration and Preprocessing

We aim to create a model capable of predicting if a customer will end his account on a bank or not. Since this is inherently a binary classification task, a logistic regressor is a great candidate for it.

After loading our churn dataset into a PySpark dataframe, we will print it and count the number of instances to get a notion of our data. With 10,000 instances, a reasonable number for the task, we will not need data augmentation. There are 10 predictor variables and one response variable: "Exited".

Looking at the response variable for the 20 records printed, it is already possible to see that there is class imbalance in this dataset. This will lead us to two scenarios: in the first one, we will not compensate for class imbalance; in the second one, we will fix the class imbalance.

In [3]:
spark = SparkSession.builder.appName("customer_churn_logistic_regression").getOrCreate()

churn = spark.read.load('../data/raw/churn.csv', format='csv', header=True,
                        inferSchema=True, sep=';')
print("Number of instances in Churn dataset: ", churn.count())
churn.show()

Number of instances in Churn dataset:  10000
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
|        645|    Spain|  Male| 44|     8|11375578| 

Our dataset has two string columns that need to be transformed to numeric before we can use them in regression. We use PySpark's `StringIndexer` for that.

With data being fully numeric, we can create a correlation matrix to check for variables correlated with the response variable or for correlation between predictor variables.

In [4]:
# Create index using the string values of Geography and Gender columns.
indexer = StringIndexer(inputCols=['Geography', 'Gender'],
                        outputCols=['GeographyIndex', 'GenderIndex'])
churn = indexer.fit(churn).transform(churn)
churn = churn.drop('Geography', 'Gender')

# Assemble every column into a single vector to be input to PySpark's
# Correlation class.
assembler = VectorAssembler(inputCols=churn.columns, outputCol='corr_features')
churn_assembled = assembler.transform(churn).select('corr_features')

# Get correlation values and build a new dataframe to work as a
# correlation matrix. 
corr_matrix = Correlation.corr(churn_assembled, 'corr_features')
corr_matrix = corr_matrix.collect()[0][corr_matrix.columns[0]].toArray()
corr_matrix = spark.createDataFrame(corr_matrix.tolist(), churn.columns)

# Every correlation value is rounded for ease of view.
corr_matrix.select([round(c, 3).alias(c) for c in corr_matrix.columns]).show()

+-----------+------+------+-------+-------------+---------+--------------+---------------+------+--------------+-----------+
|CreditScore|   Age|Tenure|Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|GeographyIndex|GenderIndex|
+-----------+------+------+-------+-------------+---------+--------------+---------------+------+--------------+-----------+
|        1.0|-0.004| 0.001|  0.007|        0.012|   -0.005|         0.026|         -0.001|-0.027|         0.008|      0.003|
|     -0.004|   1.0| -0.01|  0.022|       -0.031|   -0.012|         0.085|         -0.015| 0.285|         0.023|      0.028|
|      0.001| -0.01|   1.0| -0.017|        0.013|    0.023|        -0.028|          0.006|-0.014|         0.004|     -0.015|
|      0.007| 0.022|-0.017|    1.0|       -0.276|   -0.011|        -0.011|          0.006| 0.106|         0.063|     -0.007|
|      0.012|-0.031| 0.013| -0.276|          1.0|    0.003|          0.01|          0.014|-0.048|         0.004|      0.022|


The main correlations we can see in the matrix happen between variables: "Age" and "Exited" (0.285); "Balance" and "NumOfProducts" (-0.276); and "IsActiveMember" and "Exited" (-0.156). There is no significant correlation between independent variables. This means that they are indeed independent on each other. For this reason, and for the fact that the correlations with the dependent variable are also not very significant, we need to be careful before discarding any variables.

For now, we will not discard any variables. Still, that remains a possibility if results are bad.

PySpark's models need to take all independent variables together in a single vector. We use the R-language-like `RFormula` to perform this casting. Then, we randomly split the data between training and testing sets.

In [5]:
# Exited as dependent variable and every other as independent.
r_formula = RFormula(formula="Exited ~ .")
churn_rf = r_formula.fit(churn).transform(churn)
churn_rf.select('features', 'label').show(truncate=False)

churn_train, churn_test = churn_rf.randomSplit([0.7, 0.3])
print("Number of training instances: ", churn_train.count())
print("Number of testing instances: ", churn_test.count())

+------------------------------------------------------------+-----+
|features                                                    |label|
+------------------------------------------------------------+-----+
|[619.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7,0.0,1.0]        |1.0  |
|[608.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7,2.0,1.0]  |0.0  |
|[502.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7,0.0,1.0]  |1.0  |
|[699.0,39.0,1.0,0.0,2.0,0.0,0.0,9382663.0,0.0,1.0]          |0.0  |
|[850.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0,2.0,1.0]   |0.0  |
|[645.0,44.0,8.0,1.1375578E7,2.0,1.0,0.0,1.4975671E7,2.0,0.0]|1.0  |
|[822.0,50.0,7.0,0.0,2.0,1.0,1.0,100628.0,0.0,0.0]           |0.0  |
|[376.0,29.0,4.0,1.1504674E7,4.0,1.0,0.0,1.1934688E7,1.0,1.0]|1.0  |
|[501.0,44.0,4.0,1.4205107E7,2.0,0.0,1.0,749405.0,0.0,0.0]   |0.0  |
|[684.0,27.0,2.0,1.3460388E7,1.0,1.0,1.0,7172573.0,0.0,0.0]  |0.0  |
|[528.0,31.0,6.0,1.0201672E7,2.0,0.0,0.0,8018112.0,0.0,0.0]  |0.0  |
|[497.0,24.0,3.0,0.0,2.0,1.0,0.0,7

## Model Building and Training

We instantiate a PySpark logistic regressor using default configurations. This means that the model will have 0.5 as threshold for classifying its probabilistic output; a maximum of 100 iterations during optimization; convergence tolerance of 1e-6, i.e., loss changes smaller than 1e-6 result in end of training; no weights for different classes; and other minor settings that can be checked on the [docs](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.classification.LogisticRegression.html).

After training the model, a series of statistics can be obtained from the `summary` class attribute to display the model performance on training data. Our model achieves good results for all metrics viewed. Note that accuracy is not a good metric for measuring the performance on imbalanced datasets. Therefore, weighted precision, recall and area under the ROC curve (AUCROC) are metrics that deserve more attention.

In the context of our application, detecting customer churn for banks, it is essential to predict as many churns as possible. This way, the bank can perform countermeasures to mitigate loss of customers. Being sensitive by having a low number of false negatives, therefore reaching a high recall, is the priority. Being precise by having a low number of false positives, and a high precision, comes second. **Weighted recall and AUCROC are our focus for this problem.** They are metrics more related to sensitivity. 

In [6]:
logistic_regressor = LogisticRegression()
model = logistic_regressor.fit(churn_train)

summary = model.summary
print("Model evaluation on training set:")
print("Accuracy: ", summary.accuracy)
print("Weighted precision: ", summary.weightedPrecision)
print("Weighted recall: ", summary.weightedRecall)
print("Area under the ROC curve: ", summary.areaUnderROC)

Model evaluation on training set:
Accuracy:  0.8089407744874715
Weighted precision:  0.7755397972643587
Weighted recall:  0.8089407744874715
Area under the ROC curve:  0.7555182326233648


For the testing set, the probabilities are on the table below. There is a bias in our model towards predicting no churn (class 0). This is expected since the dataset is imbalanced towards class 0. Our model's performance is impacted by this issue and this reflects on our metrics.

Nonetheless, for the performance measures on testing data, we see results similar to those of the training set, indicating good learning generalization.

There are two new metrics with reported, F1-score and area under the precision-recall curve (AUCPR). Both are important but AUCROC and weighted recall still need more priority as they focus more on recall (sensitivity).

Note that weighted precision and weighted recall take the overall precision and recall measures and weighs them considering the class distribution of the data.

In [7]:
pred = model.transform(churn_test)
pred.select('label', 'prediction', 'probability', 'rawPrediction').show(truncate=False)

+-----+----------+----------------------------------------+------------------------------------------+
|label|prediction|probability                             |rawPrediction                             |
+-----+----------+----------------------------------------+------------------------------------------+
|1.0  |0.0       |[0.7870967416486233,0.2129032583513767] |[1.3075132890562,-1.3075132890562]        |
|1.0  |0.0       |[0.79610294879415,0.20389705120585]     |[1.3621132946242502,-1.3621132946242502]  |
|1.0  |0.0       |[0.860734569830777,0.139265430169223]   |[1.8214044938510021,-1.8214044938510021]  |
|1.0  |0.0       |[0.8278527405299569,0.17214725947004306]|[1.570485018739304,-1.570485018739304]    |
|1.0  |0.0       |[0.8069897389231682,0.19301026107683183]|[1.4305675994072864,-1.4305675994072864]  |
|1.0  |0.0       |[0.7745402044110802,0.22545979558891982]|[1.2341277156836554,-1.2341277156836554]  |
|1.0  |0.0       |[0.9275317176435545,0.07246828235644553]|[2.54937800941

In [8]:
binary_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
print("Model evaluation on testing set:")
print("Area under the ROC curve:", binary_evaluator.evaluate(pred))
binary_evaluator.setMetricName('areaUnderPR')
print("Area under the PR curve:", binary_evaluator.evaluate(pred))

multiclass_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
print("Accuracy:", multiclass_evaluator.evaluate(pred))
multiclass_evaluator.setMetricName('weightedPrecision')
print("Weighted precision:", multiclass_evaluator.evaluate(pred))
multiclass_evaluator.setMetricName('weightedRecall')
print("Weighted recall:", multiclass_evaluator.evaluate(pred))
multiclass_evaluator.setMetricName('f1')
print("F1-score:", multiclass_evaluator.evaluate(pred))

Model evaluation on testing set:
Area under the ROC curve: 0.754717290402596
Area under the PR curve: 0.44706285563820014
Accuracy: 0.8020833333333334
Weighted precision: 0.7690886987237457
Weighted recall: 0.8020833333333334
F1-score: 0.7540965467365904


## Second Scenario: Compensating for Class Imbalance

Since our results are far from perfect, we have to go further with our analysis. The two most clear options to follow are:
1. Perform feature selection by dropping independent variables less correlated with the dependent variable;
2. Account for class imbalance.

The table below shows that, as we observed before, class imbalance in this dataset is significant. There are 4 times more elements of class 0 than elements of class 1. This made our first model have a bias for class 0. As class imbalance is much more significant than variable correlation, we go with the second option.

In [9]:
churn.groupBy('Exited').count().show()

+------+-----+
|Exited|count|
+------+-----+
|     1| 2037|
|     0| 7963|
+------+-----+



We have two options for handling class imbalance:
1. Resample the dataset so that we end up with the same amount of class 0 and class 1 elements in our training and testing sets;
2. Attribute weights for each class before training so that the minority class gets more attention from the model.

Fortunately for us, logistic regression can easily work with class weights and PySpark's regression models support this. Option 2 is also preferred because we can avoid having to simulate data for the minority class or having to undersample the majority class. In the end, we still work with the original dataset.

The "ClassWeightCol", presented in the next table, will provide the weights for each element to our new logistic regressor. Now, class 0 will not overshadow class 1.

In [10]:
# Get the probability for any element to be part of class 1.
data_balancing_ratio = (churn.where(churn.Exited == 1).count()
                       /  churn.count())

# Define an UDF to be used as a dataframe function when providing class
# weights. As this a binary classification problem, we can work with the
# previous probability and its complement.
calculate_weights = udf(lambda x: 1 * data_balancing_ratio if x == 0
                        else (1 * (1.0 - data_balancing_ratio)), DoubleType())
                        
weighted_churn = churn.withColumn('ClassWeightCol', calculate_weights('Exited'))
weighted_churn.show()

+-----------+---+------+--------+-------------+---------+--------------+---------------+------+--------------+-----------+--------------+
|CreditScore|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|GeographyIndex|GenderIndex|ClassWeightCol|
+-----------+---+------+--------+-------------+---------+--------------+---------------+------+--------------+-----------+--------------+
|        619| 42|     2|       0|            1|        1|             1|       10134888|     1|           0.0|        1.0|        0.7963|
|        608| 41|     1| 8380786|            1|        0|             1|       11254258|     0|           2.0|        1.0|        0.2037|
|        502| 42|     8| 1596608|            3|        1|             0|       11393157|     1|           0.0|        1.0|        0.7963|
|        699| 39|     1|       0|            2|        0|             0|        9382663|     0|           0.0|        1.0|        0.2037|
|        850| 43|     2|12551082| 

In [11]:
weighted_churn_rf = r_formula.fit(weighted_churn).transform(weighted_churn)
weighted_churn_rf.select('features', 'label').show(truncate=False)

weighted_churn_train, weighted_churn_test = weighted_churn_rf.randomSplit([0.7, 0.3])
print("Number of training instances: ", weighted_churn_train.count())
print("Number of testing instances: ", weighted_churn_test.count())

+-------------------------------------------------------------------+-----+
|features                                                           |label|
+-------------------------------------------------------------------+-----+
|[619.0,42.0,2.0,0.0,1.0,1.0,1.0,1.0134888E7,0.0,1.0,0.7963]        |1.0  |
|[608.0,41.0,1.0,8380786.0,1.0,0.0,1.0,1.1254258E7,2.0,1.0,0.2037]  |0.0  |
|[502.0,42.0,8.0,1596608.0,3.0,1.0,0.0,1.1393157E7,0.0,1.0,0.7963]  |1.0  |
|[699.0,39.0,1.0,0.0,2.0,0.0,0.0,9382663.0,0.0,1.0,0.2037]          |0.0  |
|[850.0,43.0,2.0,1.2551082E7,1.0,1.0,1.0,790841.0,2.0,1.0,0.2037]   |0.0  |
|[645.0,44.0,8.0,1.1375578E7,2.0,1.0,0.0,1.4975671E7,2.0,0.0,0.7963]|1.0  |
|[822.0,50.0,7.0,0.0,2.0,1.0,1.0,100628.0,0.0,0.0,0.2037]           |0.0  |
|[376.0,29.0,4.0,1.1504674E7,4.0,1.0,0.0,1.1934688E7,1.0,1.0,0.7963]|1.0  |
|[501.0,44.0,4.0,1.4205107E7,2.0,0.0,1.0,749405.0,0.0,0.0,0.2037]   |0.0  |
|[684.0,27.0,2.0,1.3460388E7,1.0,1.0,1.0,7172573.0,0.0,0.0,0.2037]  |0.0  |
|[528.0,31.0

Looking at the same metrics we saw for the training phase of the first model, we already note a huge boost in performance. All metrics are perfect, even weighted recall and AUCROC, our prioritized metrics. This indicates that the main challenge for this data really is class imbalance. A logistic regressor, even being a simple model, seems to be perfectly able to understand this dataset. Now we check the testing results to confirm this notion and that the model has good generalization capabilities.

In [12]:
logistic_regressor.setWeightCol('ClassWeightCol')
model = logistic_regressor.fit(weighted_churn_train)

summary = model.summary
print("Model evaluation on training set:")
print("Accuracy: ", summary.accuracy)
print("Weighted precision: ", summary.weightedPrecision)
print("Weighted recall: ", summary.weightedRecall)
print("Area under the ROC curve: ", summary.areaUnderROC)

Model evaluation on training set:
Accuracy:  1.0
Weighted precision:  1.0
Weighted recall:  1.0
Area under the ROC curve:  0.999999483724412


In the table below, for the 20 samples shown, our model does not commit a single mistake. All instances are predicted with above 99% probability and with much larger values for raw prediction, also indicating that the model is more certain of its predictions.

In [13]:
pred = model.transform(weighted_churn_test)
pred.select('label', 'prediction', 'probability', 'rawPrediction').show(truncate=False)

+-----+----------+------------------------------------------+----------------------------------------+
|label|prediction|probability                               |rawPrediction                           |
+-----+----------+------------------------------------------+----------------------------------------+
|1.0  |1.0       |[3.4080155213727247E-9,0.9999999965919845]|[-19.497135670188293,19.497135670188293]|
|1.0  |1.0       |[2.879363958860322E-9,0.999999997120636]  |[-19.665696411927563,19.665696411927563]|
|1.0  |1.0       |[3.5289505204045916E-9,0.9999999964710495]|[-19.462265309677086,19.462265309677086]|
|1.0  |1.0       |[5.337068853688215E-9,0.9999999946629311] |[-19.048589233143787,19.048589233143787]|
|1.0  |1.0       |[6.915837366589959E-9,0.9999999930841627] |[-18.789451778016684,18.789451778016684]|
|1.0  |1.0       |[4.754471197700219E-9,0.9999999952455288] |[-19.16418035211592,19.16418035211592]  |
|1.0  |1.0       |[7.3168874515820805E-9,0.9999999926831126]|[-18.7330808

For all performance measures taken, our model achieves an almost perfect score. This means that indeed the main difficulty our previous model had was dealing with imbalanced data. Our prioritized metrics, AUCROC and weighted recall are very high. Other metrics that involve recall, such as AUCPR and F1-score, are also top-notch.

Our other option for furthering the analysis was to perform feature selection. However, since our performance is as good as it can be, we will not follow that path.

We conclude that the final model is able to predict if customers are going to close their account on the bank with great precision and accuracy. Especially considering a dataset with reasonable class balance or if proper class weights are supplied to the model.

In [14]:
binary_evaluator = BinaryClassificationEvaluator(metricName='areaUnderROC')
print("Model evaluation on testing set:")
print("Area under the ROC curve:", binary_evaluator.evaluate(pred))
binary_evaluator.setMetricName('areaUnderPR')
print("Area under the PR curve:", binary_evaluator.evaluate(pred))

multiclass_evaluator = MulticlassClassificationEvaluator(metricName='accuracy')
print("Accuracy:", multiclass_evaluator.evaluate(pred))
multiclass_evaluator.setMetricName('weightedPrecision')
print("Weighted precision:", multiclass_evaluator.evaluate(pred))
multiclass_evaluator.setMetricName('weightedRecall')
print("Weighted recall:", multiclass_evaluator.evaluate(pred))
multiclass_evaluator.setMetricName('f1')
print("F1-score:", multiclass_evaluator.evaluate(pred))

Model evaluation on testing set:
Area under the ROC curve: 0.999999353476004
Area under the PR curve: 0.9999975699962578
Accuracy: 1.0
Weighted precision: 1.0
Weighted recall: 1.0
F1-score: 1.0
